In [1]:
import warnings
import numpy as np
import pandas as pd 

from sklearn.preprocessing import LabelEncoder, MinMaxScaler

from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neural_network import MLPClassifier

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from src.wrapped import Wrapped

In [2]:
warnings.filterwarnings("ignore")

wp = Wrapped(
    '../data/row/',
    '../data/processed/',
    '../data/files/'
)

# Importando dados para treinamento

In [ ]:
df = wp.load_data('df_inst_aerofonerumentos')
df

- Algumas informações

In [ ]:
print('Dimensões do dataframe', df.shape)
print('Total de instrumentos na base original:', len(df.instrumento.unique()))
df.instrumento.value_counts()

# Preprocessamento

In [ ]:
le = LabelEncoder()

df['labels'] = le.fit_transform(df.instrumento)

# Treinando modelos

In [ ]:
X = df.drop(columns=['labels', 'file_name'])
y = df['labels']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

In [ ]:
instrumentos_test = X_test['instrumento']
proporcao_dados_instrumentos_treino = X_train.instrumento.value_counts()

X_train = X_train.drop(columns=['instrumento'])
X_test = X_test.drop(columns=['instrumento'])

- Baseline

In [ ]:
models = np.array([KNeighborsClassifier(),DecisionTreeClassifier(), RandomForestClassifier(), LGBMClassifier(),MLPClassifier()])

In [ ]:
for model in models:
    model.fit(X_train, y_train)
    predict = model.predict(X_test)
    print(f'Acuracia {model}: {accuracy_score(y_test, predict).round(2) * 100}%')
    

# Vamos analisar esses resultados

In [ ]:
model = RandomForestClassifier()
model.fit(X_train, y_train)
predict = model.predict(X_test)
print(f'Acuracia Random Forest: {accuracy_score(y_test, predict).round(2) * 100}%')

In [ ]:
# criar um dataframe de gabarito para auxiliar nas nossas analises
gabarito = X_test
gabarito['labels'] = y_test
gabarito['instrumentos'] = instrumentos_test

# filtrando os acertos
acertos = gabarito[predict == gabarito.labels]

# filtrando os erros
erros = gabarito[predict != gabarito.labels]

- Acerto total

In [ ]:
print('Total de acertos: ', acertos.shape[0])

- Total de instrumentos indentificados

In [ ]:
len(acertos.instrumentos.unique())

- Instrumentos não indentificados

mandolin e clarinete

- Proporção de dados por instrumentos usado no treino

In [ ]:
proporcao_dados_instrumentos_treino

- Proporção de instrumentos no Test

In [ ]:
gabarito.instrumentos.value_counts()

- Proporção de acertos por instrumentos

In [ ]:
acertos.instrumentos.value_counts()

- Proporção de erros por instrumentos

In [ ]:
erros.instrumentos.value_counts()